In [1]:
import gc
import sys
import time
import pickle
import random
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier

In [2]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 7) 
plt.style.use('ggplot')

In [3]:
transactions = pd.read_csv('../../data/transactions.csv')

In [4]:
# топ сочетаний mcc и cat (см README.md)
m = pd.read_csv('../1_cv/mcc_matches.csv')
m = m[(m['clients'] > 20)][:400]
m['mcc_code'] = m['mcc_code'].astype(int)
m['cat_id'] = m['cat_id'].astype(int)
mcc = list(m['mcc_code'])
cat = list(m['cat_id'])

In [5]:
m.head()

,mcc_code,cat_id,dist,count,clients
2941,7832,552,6504,35,33
3132,5200,170,6801,80,27
3211,4784,794,6888,40,27
3305,5999,606,6999,25,23
3580,5815,763,7325,40,22


In [6]:
transactions = transactions[transactions['mcc_code'].isin(mcc)]
transactions['dt'] = transactions['transaction_dttm'].str[:10]

In [7]:
transactions.head()

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm,dt
0,000932580e404dafbecd5916d4640938,5411,48,-361.07230,2020-08-03 08:05:23,2020-08-03
1,000932580e404dafbecd5916d4640938,5499,48,-137.31398,2020-08-05 01:27:40,2020-08-05
2,000932580e404dafbecd5916d4640938,5499,48,-138.84981,2020-08-05 03:28:11,2020-08-05
3,000932580e404dafbecd5916d4640938,4829,48,-309.47653,2020-08-06 00:36:29,2020-08-06
4,000932580e404dafbecd5916d4640938,5411,48,-133.47370,2020-08-09 00:30:13,2020-08-09


In [8]:
clickstream = pd.read_csv('../../data/clickstream.csv')

In [9]:
clickstream = clickstream[clickstream['cat_id'].isin(cat)]
clickstream['dt'] = clickstream['timestamp'].str[:10]

In [11]:
gc.collect()

73

In [13]:
puz = pd.read_csv('../../data_mini/puzzle.csv')

In [14]:
puz.head()

,bank,rtk
0,6dd66e8624da427da6b558903a5772b8,56f386b1a1a9455b9118cd290b0627d7
1,224a2325b44a4326bc539e3f1a6e713b,ca77a88196ef4dd786fb390340cf226e
2,ed547c635e594b88a9eb5b5f7ae75304,0ca257d60e0a47c085117c80dff13cb5
3,10f09bfecc7f4cf894897206d4020307,3c044d4b126f439886f0f7aee8b16545
4,8a1dd91a260143f4b6044da26844dde2,c07ebeb15f7d43b6b6450f7c50b8f245


In [15]:
transactions = transactions[transactions['user_id'].isin(list(puz['bank']))]
clickstream = clickstream[clickstream['user_id'].isin(list(puz['rtk']))]

In [16]:
# берем пересекающиеся даты
tset = set(transactions['dt'].unique())
cset = set(clickstream['dt'].unique())
intdt = cset.intersection(tset)

In [17]:
transactions = transactions[transactions['dt'].isin(intdt)]
clickstream = clickstream[clickstream['dt'].isin(intdt)]

In [18]:
print(transactions.shape[0])
print(clickstream.shape[0])

2303085
31905214


In [19]:
# кол-во минут с начала суток
transactions['sec1'] = transactions['transaction_dttm'].str[11:13].astype(int)*60 + \
                       transactions['transaction_dttm'].str[14:16].astype(int) + 1

In [20]:
# кол-во минут с начала суток
clickstream['sec2'] = clickstream['timestamp'].str[11:13].astype(int)*60 + \
                       clickstream['timestamp'].str[14:16].astype(int) + 1

In [21]:
bankclient_embed = transactions.pivot_table(index = ['user_id', 'dt'],
                            values=['sec1'],
                            columns=['mcc_code'],
                            aggfunc=['median']).fillna(0)
bankclient_embed.columns = ['v1_' + str(i[0]) + '-' + str(i[2]) for i in bankclient_embed.columns]

In [23]:
bankclient_embed.head()

v1_median--1  v1_median-742  \
user_id                          dt                                        
001708a5fca04c2ca58d7188b747d5f5 2021-01-17           0.0            0.0   
                                 2021-01-18           0.0            0.0   
                                 2021-01-19           0.0            0.0   
                                 2021-01-20           0.0            0.0   
                                 2021-01-21           0.0            0.0   

                                             v1_median-780  v1_median-1711  \
user_id                          dt                                          
001708a5fca04c2ca58d7188b747d5f5 2021-01-17            0.0             0.0   
                                 2021-01-18            0.0             0.0   
                                 2021-01-19            0.0             0.0   
                                 2021-01-20            0.0             0.0   
                                 2021-01-21            0.0             0.0   

                                             v1_median-3011  v1_median-4111  \
user_id                          dt                                           
001708a5fca04c2ca58d7188b747d5f5 2021-01-17             0.0             0.0   
                                 2021-01-18             0.0             0.0   
                                 2021-01-19             0.0             0.0   
                                 2021-01-20             0.0             0.0   
                                 2021-01-21             0.0             0.0   

                                             v1_median-4112  v1_median-4121  \
user_id                          dt                                           
001708a5fca04c2ca58d7188b747d5f5 2021-01-17             0.0             0.0   
                                 2021-01-18             0.0             0.0   
                                 2021-01-19             0.0             0.0   
                                 2021-01-20             0.0             0.0   
                                 2021-01-21           249.0             0.0   

                                             v1_median-4131  v1_median-4214  \
user_id                          dt                                           
001708a5fca04c2ca58d7188b747d5f5 2021-01-17             0.0             0.0   
                                 2021-01-18             0.0             0.0   
                                 2021-01-19             0.0             0.0   
                                 2021-01-20             0.0             0.0   
                                 2021-01-21             0.0             0.0   

                                             ...  v1_median-8099  \
user_id                          dt          ...                   
001708a5fca04c2ca58d7188b747d5f5 2021-01-17  ...             0.0   
                                 2021-01-18  ...             0.0   
                                 2021-01-19  ...             0.0   
                                 2021-01-20  ...             0.0   
                                 2021-01-21  ...             0.0   

                                             v1_median-8111  v1_median-8220  \
user_id                          dt                                           
001708a5fca04c2ca58d7188b747d5f5 2021-01-17             0.0             0.0   
                                 2021-01-18             0.0             0.0   
                                 2021-01-19             0.0             0.0   
                                 2021-01-20             0.0             0.0   
                                 2021-01-21             0.0             0.0   

                                             v1_median-8398  v1_median-8699  \
user_id                          dt                                           
001708a5fca04c2ca58d7188b747d5f5 2021-01-17             0.0             0.0   
                                 20

In [24]:
del transactions
gc.collect()

132

In [25]:
clickstream_embed = clickstream.pivot_table(index = ['user_id', 'dt'],
                        values=['sec2'],
                        columns=['cat_id'],
                        aggfunc=['median']).fillna(0)

clickstream_embed.columns = ['v2_' + str(i[0]) + '-' + str(i[2]) for i in clickstream_embed.columns]

In [26]:
clickstream_embed.head()

v2_median-1  v2_median-11  \
user_id                          dt                                      
000fc5be9e974980a1c9728f4866db11 2021-01-19          0.0           0.0   
                                 2021-01-22          0.0           0.0   
                                 2021-01-23          0.0           0.0   
                                 2021-01-25          0.0           0.0   
                                 2021-01-26          0.0           0.0   

                                             v2_median-12  v2_median-13  \
user_id                          dt                                       
000fc5be9e974980a1c9728f4866db11 2021-01-19           0.0           0.0   
                                 2021-01-22           0.0           0.0   
                                 2021-01-23           0.0           0.0   
                                 2021-01-25           0.0           0.0   
                                 2021-01-26           0.0           0.0   

                                             v2_median-14  v2_median-15  \
user_id                          dt                                       
000fc5be9e974980a1c9728f4866db11 2021-01-19           0.0           0.0   
                                 2021-01-22           0.0           0.0   
                                 2021-01-23           0.0           0.0   
                                 2021-01-25           0.0           0.0   
                                 2021-01-26           0.0           0.0   

                                             v2_median-19  v2_median-20  \
user_id                          dt                                       
000fc5be9e974980a1c9728f4866db11 2021-01-19           0.0           0.0   
                                 2021-01-22           0.0           0.0   
                                 2021-01-23           0.0           0.0   
                                 2021-01-25           0.0           0.0   
                                 2021-01-26           0.0           0.0   

                                             v2_median-21  v2_median-25  ...  \
user_id                          dt                                      ...   
000fc5be9e974980a1c9728f4866db11 2021-01-19           0.0           0.0  ...   
                                 2021-01-22           0.0           0.0  ...   
                                 2021-01-23           0.0           0.0  ...   
                                 2021-01-25           0.0         684.0  ...   
                                 2021-01-26           0.0           0.0  ...   

                                             v2_median-1455  v2_median-1456  \
user_id                          dt                                           
000fc5be9e974980a1c9728f4866db11 2021-01-19             0.0             0.0   
                                 2021-01-22             0.0             0.0   
                                 2021-01-23             0.0             0.0   
                                 2021-01-25             0.0             0.0   
                                 2021-01-26             0.0             0.0   

                                             v2_median-1457  v2_median-1458  \
user_id                          dt                                           
000fc5be9e974980a1c9728f4866db11 2021-01-19             0.0             0.0   
                                 2021-01-22             0.0             0.0   
                                 2021-01-23             0.0             0.0   
                                 2021-01-25           156.0             0.0   
                                 2021-01-26             0.0             0.0   

                                             v2_median-1459  v2_median-1522  \
user_id                          dt                                           
000fc5be9e974980a1c9728f4866db11 2021-01-19             0.0             0.0   
                                 2021-01-2

In [27]:
del clickstream
gc.collect()

40

In [28]:
def df_compress(df):
    dtype_df = list()
    for x in df.dtypes.tolist():
        if x=='int64':
            dtype_df.append('int32')
        elif(x=='float64'):
            dtype_df.append('int32')
        elif(x=='float32'):
            dtype_df.append('int32')
        elif(x=='int32'):
            dtype_df.append('int32')
        else:
            dtype_df.append('object')
    
    dtype_df = dict(zip(df.columns.tolist(),dtype_df))
    df = df.astype(dtype_df)
    return df

In [29]:
clickstream_embed = df_compress(clickstream_embed)

In [30]:
bankclient_embed = df_compress(bankclient_embed)

In [44]:
bankclient_embed = bankclient_embed.reset_index()
clickstream_embed = clickstream_embed.reset_index()

In [47]:
bank_ids = list(bankclient_embed['user_id'].unique())
rtk_ids = list(clickstream_embed['user_id'].unique())

In [50]:
gc.collect()

280

### Model

In [38]:
clf = CatBoostClassifier()
clf.load_model('../models/v5_daily_cross_nearest.cbm',  format='cbm')

In [51]:
clickstream_embed.rename(columns={'user_id': 'rtk'}, inplace=True)
clickstream_embed.set_index(['rtk', 'dt'], inplace=True)

In [52]:
bankclient_embed.rename(columns={'user_id': 'bank'}, inplace=True)
bankclient_embed.set_index(['bank'], inplace=True)

### Predict

In [55]:
%%time

D = pd.DataFrame()
pack = 5
k = len(bank_ids) // pack

for jj in range(k+1):
    this_bank_ids = bank_ids[jj*pack:(jj+1)*pack]
    ps = pd.DataFrame(this_bank_ids, columns=['bank'])
    ps['rtk'] = ps['bank'].apply(lambda x: rtk_ids)
    ps = ps.explode('rtk')
    
    ps.set_index(['bank'], inplace=True)
    
    be = bankclient_embed[bankclient_embed.index.isin(this_bank_ids)]
    ps = be.join(ps, how='left')
    
    ps = ps.reset_index()
    ps = ps.set_index(['rtk', 'dt'])
    
    ps = ps.join(clickstream_embed, how='inner', on=['rtk', 'dt'])
    
    # основная магия происходит тут
    for i in range(len(mcc)):
        ps[str(int(mcc[i])) + '_' + str(int(cat[i]))] = np.sign(ps['v1_median-' + str(int(mcc[i]))]) * \
                                                        np.sign(ps['v2_median-' + str(int(cat[i]))]) * \
                        abs(ps['v1_median-' + str(int(mcc[i]))] - ps['v2_median-' + str(int(cat[i]))])
        
    ps = ps.reset_index()
    
    P = ps[['bank', 'rtk', 'dt']].copy()
    P['pred'] = clf.predict_proba(ps.drop(['bank', 'rtk', 'dt'] + to_drop_cols, axis=1))[:,1]
    
    # за каждую дату оставляем топ 100 кандидатов
    P = P.sort_values(['bank', 'pred'], ascending=False)
    P = P.groupby(['bank', 'dt']).head(100)
    
    D = D.append(P, ignore_index=True)
    print(jj, P.shape[0], P['bank'].nunique())
    del P
    del ps
    gc.collect()

0 72939 5
1 70326 5
2 71265 5
3 76813 5
4 85739 5
5 76652 5
6 66239 5
7 85852 5
8 68539 5
9 69752 5
10 70839 5
11 88452 5
12 59826 5
13 77739 5
14 64639 5
15 72339 5
16 79339 5
17 70365 5
18 67252 5
19 73739 5
20 63939 5
21 81665 5
22 85852 5
23 79826 5
24 76652 5
25 67326 5
26 75339 5
27 82652 5
28 62939 5
29 74839 5
30 78765 5
31 64052 5
32 80926 5
33 81839 5
34 71452 5
35 71039 5
36 74939 5
37 66352 5
38 65452 5
39 74026 5
40 77239 5
41 87939 5
42 81552 5
43 59439 5
44 77126 5
45 72552 5
46 73252 5
47 71226 5
48 79539 5
49 73739 5
50 72239 5
51 76152 5
52 77452 5
53 70265 5
54 76665 5
55 78552 5
56 81539 5
57 79965 5
58 69852 5
59 69239 5
60 73313 5
61 77252 5
62 76852 5
63 70639 5
64 76339 5
65 64665 5
66 78952 5
67 74826 5
68 76839 5
69 80239 5
70 74065 5
71 77552 5
72 82239 5
73 69800 5
74 73452 5
75 79052 5
76 77052 5
77 83952 5
78 72565 5
79 62626 5
80 69952 5
81 82065 5
82 75639 5
83 75552 5
84 72765 5
85 73452 5
86 75152 5
87 74752 5
88 78039 5
89 78926 5
90 77465 5
91 67539 

692 77352 5
693 71565 5
694 83652 5
695 82065 5
696 79739 5
697 78639 5
698 66265 5
699 77639 5
700 74839 5
701 65826 5
702 73352 5
703 79865 5
704 70852 5
705 67439 5
706 78739 5
707 75952 5
708 78252 5
709 75152 5
710 79939 5
711 81639 5
712 75439 5
713 75526 5
714 73252 5
715 67352 5
716 77339 5
717 74665 5
718 66739 5
719 73852 5
720 75852 5
721 75326 5
722 61839 5
723 70239 5
724 69752 5
725 70465 5
726 66065 5
727 81765 5
728 60139 5
729 70913 5
730 77339 5
731 77752 5
732 72665 5
733 66665 5
734 86352 5
735 79526 5
736 78652 5
737 82265 5
738 68352 5
739 64752 5
740 74752 5
741 75739 5
742 69339 5
743 64639 5
744 89465 5
745 74552 5
746 83239 5
747 72826 5
748 81465 5
749 75852 5
750 85965 5
751 69039 5
752 69126 5
753 82052 5
754 70939 5
755 77039 5
756 79852 5
757 76265 5
758 70026 5
759 78165 5
760 70865 5
761 78652 5
762 74452 5
763 78265 5
764 76739 5
765 76739 5
766 77652 5
767 76952 5
768 86439 5
769 79965 5
770 75552 5
771 63526 5
772 74452 5
773 78252 5
774 77439 5
775 

In [56]:
gc.collect()

20

In [3]:
# save data
#D.to_csv('daily_cross_predictions_nearest_Data_all_clients.csv', index=None)
#D = pd.read_csv('daily_cross_predictions_nearest_Data_all_clients.csv')

In [4]:
D.head()

,bank,rtk,dt,pred
0,006415c4240844e0810617ecebbd2db4,26b3ac29b3584b2f807ce404546d9b23,2021-06-26,0.468148
1,006415c4240844e0810617ecebbd2db4,8b918d5da8a8408bb690c9e04ed16e5c,2021-03-06,0.431815
2,006415c4240844e0810617ecebbd2db4,8b21f9d6b4884a3d9e89ca5d434ee2da,2021-06-26,0.428346
3,006415c4240844e0810617ecebbd2db4,c5e93edb6f914acf83b2a3e0bec43b66,2021-03-06,0.422050
4,006415c4240844e0810617ecebbd2db4,15fb49d4823940ebb5d7adeacd8c3391,2021-06-26,0.421368


In [7]:
# cross_v3 - D['pred_h'] = D['pred']
# cross_v2 - D['pred_h'] = 2 * D['pred'] * D['pred_scaled2'] / (D['pred'] + D['pred_scaled2'])
# cross_v1_4 - S['pred_h'] = 3 * S['t1'] * S['t3'] * S['t2'] / (S['t1'] * S['t3'] + S['t1'] * S['t2'] + S['t2'] * S['t3'])
#              D['pred_h'] = 2 * D['pred'] * D['pred_scaled2'] / (D['pred'] + D['pred_scaled2'])

# cross_v1_5 - S['pred_h'] = 3 * S['t1'] * S['t3'] * S['t2'] / (S['t1'] * S['t3'] + S['t1'] * S['t2'] + S['t2'] * S['t3'])
#              D['pred_h'] = 2 * D['pred'] * D['pred_scaled3'] / (D['pred'] + D['pred_scaled3'])

# cross_v1_5 - S['pred_h'] = 3 * S['t1'] * S['t3'] * S['t2'] / (S['t1'] * S['t3'] + S['t1'] * S['t2'] + S['t2'] * S['t3'])
#              D['pred_h'] = 2 * D['pred'] * D['pred_scaled3'] / (D['pred'] + D['pred_scaled3'])

# cross_v1_6 - S['pred_h'] = 3 * S['t1'] * S['t3'] * S['t2'] / (S['t1'] * S['t3'] + S['t1'] * S['t2'] + S['t2'] * S['t3'])
#              D['pred_h'] = 3 * D['pred'] * D['pred_scaled2'] * D['pred_scaled3'] / (...)

# cross_v1_7 - S['pred_h'] = 3 * S['t1'] * S['t3'] * S['t2'] / (S['t1'] * S['t3'] + S['t1'] * S['t2'] + S['t2'] * S['t3'])
#              D['pred_h'] = 2 * D['pred'] * D['pred_scaled3'] / (...)


#D = nxscaler2(D, 'pred')
#D = nxscaler3(D, 'pred')
#D['pred_h'] = 2 * D['pred'] * D['pred_scaled3']/ (D['pred'] + D['pred_scaled3'])
#D['pred_h'] = 2 * D['pred'] * D['pred_scaled2']/ (D['pred'] + D['pred_scaled2'])

In [8]:
# нужно усреднить результаты. Так как за каждую дату отбирали топ 100, 
# то простое усреднение не совсем логично и дает хуже результат

S = D.groupby(['bank', 'rtk']).agg({
      'pred_h': ['sum', 'mean', 'std', 'count']
}).reset_index()
S.columns = ['bank', 'rtk', 'sum', 'mean', 'std', 'count']

S['t1'] = S['sum'] / np.sqrt(S['count'])
S['t2'] = S['sum'] * S['std']
S['t3'] = S['sum']
S['pred_h'] = 3 * S['t1'] * S['t3'] * S['t2'] / (S['t1'] * S['t3'] + S['t1'] * S['t2'] + S['t2'] * S['t3'])

S = S.sort_values(['bank', 'pred_h'], ascending=False)
S.rename(columns={'pred_h': 'pred'}, inplace=True)
S.head()

,bank,rtk,sum,mean,std,count,t1,t2,t3,pred
16016256,fff71f6ea9f54fa9a8575c772e0cdf71,e4b6ca0f374c43098557a2b579e2c1f9,3.192240,0.152011,0.195007,21,0.696604,0.622508,3.192240,0.894134
16016442,fff71f6ea9f54fa9a8575c772e0cdf71,f63a291da4f14a06bb893594feeedc90,4.784076,0.251793,0.083484,19,1.097542,0.399393,4.784076,0.827825
16016497,fff71f6ea9f54fa9a8575c772e0cdf71,fb47a315cf584dbdb1f1933ba2b6edd4,3.237699,0.179872,0.152275,18,0.763133,0.493022,3.237699,0.822469
16015697,fff71f6ea9f54fa9a8575c772e0cdf71,b4ce9e22a0f64e6d81de59500f553b88,1.718521,0.286420,0.309711,6,0.701583,0.532245,1.718521,0.771987
16015057,fff71f6ea9f54fa9a8575c772e0cdf71,79996d2ae02742aba1927f3c4ef7dc6b,1.943713,0.277673,0.255280,7,0.734654,0.496190,1.943713,0.771006


In [6]:
#minmaxscaler by group

# Еще одна находна это - minmax scaler, после её применения всегда R1 скор выше
# Логика такая - после предикта, средний скор по rtk получается перекошенным. У кого то скор высокий 0.9, а у кого то 0.5
# хотя, с одинаковой вероятность есть пара у того и у другого. Чтобы немного подправить это - применяю scaler

def nxscaler(df, col):
    g = df.groupby('rtk')[col]
    min_, max_ = g.transform('min'), g.transform('max')
    df[col+'_scaled'] = (df[col] - min_) / (max_ - min_)
    return df

# not used
def nxscaler2(df, col):
    g = df.groupby(['bank', 'dt'])[col]
    min_, max_ = g.transform('min'), g.transform('max')
    df[col+'_scaled2'] = (df[col] - min_) / (max_ - min_)
    return df

# not used
def nxscaler3(df, col):
    g = df.groupby(['rtk', 'dt'])[col]
    min_, max_ = g.transform('min'), g.transform('max')
    df[col+'_scaled3'] = (df[col] - min_) / (max_ - min_)
    return df

In [9]:
S = S.groupby(['bank']).head(200).copy()

In [10]:
P = S.copy()

In [11]:
R = P.copy()
R = nxscaler(R, 'pred')

In [12]:
col = 'pred_h'

In [13]:
# берем среднее гармоническое между скором предикта и отскейленнего предикта
# во всех сабмитах R1 скор выше, чем - без

R['pred_h'] = 2*R['pred'] * R['pred_scaled'] / (R['pred'] + R['pred_scaled'])

In [14]:
R = R.sort_values(['bank', col], ascending=False)

In [15]:
F = R.groupby('bank').head(200).copy()

In [18]:
F.head()

,bank,rtk,sum,mean,std,count,t1,t2,t3,pred,pred_scaled,pred_h
16015520,fff71f6ea9f54fa9a8575c772e0cdf71,a5757fe961ac48c1a9f79bbb4b81001c,2.412155,0.402026,0.106975,6,0.984758,22.548738,2.412155,2.034735,0.746750,1.092538
16015322,fff71f6ea9f54fa9a8575c772e0cdf71,925e1cc0e33143ffbbcbd483b4e840d1,2.741297,0.304589,0.094963,9,0.913766,28.866858,2.741297,2.008294,0.743640,1.085380
16016442,fff71f6ea9f54fa9a8575c772e0cdf71,f63a291da4f14a06bb893594feeedc90,5.168581,0.272031,0.107420,19,1.185754,48.115591,5.168581,2.836597,0.657860,1.068025
16015019,fff71f6ea9f54fa9a8575c772e0cdf71,74dbe0057e064168bd55b7f5526cfb31,0.984148,0.492074,0.160965,2,0.695898,6.114036,0.984148,1.146500,0.808477,0.948266
16016038,fff71f6ea9f54fa9a8575c772e0cdf71,d1d4e79170284c768b9fb4ca8e885cd9,1.579012,0.263169,0.207911,6,0.644629,7.594668,1.579012,1.295192,0.713607,0.920209


In [16]:
F = F[['bank', 'rtk']]
F = F.pivot_table(index='bank', values='rtk', aggfunc=list)
F = F.reset_index()
F = F[['bank', 'rtk']]
F.columns = ['bank', 'rtk_list']

In [17]:
F.head()

,bank,rtk_list
0,001708a5fca04c2ca58d7188b747d5f5,"[2c3d6475583e488ab8524aaddce5883a, fe0f062b018..."
1,0025930588d249ec8e73936f6a6d05bd,"[c1617b3c08e04f6ea37f1f902282d5ff, 65067f5d0a8..."
2,0034020d25da4951b559da1d3171a1e3,"[7136fcfc54df4d34a74ed43257fc15ec, 8b5c8d23330..."
3,00491339a09c4638a0f27ca51f615d67,"[4a907fea0b4547e5a0d7062a6a9d945d, 5f1f0fa2099..."
4,006415c4240844e0810617ecebbd2db4,"[702fd1c2fab54b01943729bd88bb16a9, c5e93edb6f9..."


In [18]:
F.to_csv('cross_v1_7_200.csv', index=None)

In [19]:
F.shape

(4952, 2)